In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.set_style("darkgrid")

In [2]:
# Importación de datos:
dataset_path = "Mate_PI_2020_2025.csv"
matepi = pd.read_csv(dataset_path, index_col=0, dtype={"Condicion":"category", "Tipo_Cursada":"category"})

# Descripción de la base de datos

Cada registro es único, corresponde a los resultados de cada alumno, identificados por el índice de fila.

- **1P1F**: nota del Primer Parcial, Primera Fecha.
- **1P2F**: nota del Primer Parcial, Segunda Fecha.
- **2P1F**: nota del Segundo Parcial, Primera Fecha.
- **2P2F**: nota del Segundo Parcial, Segunda Fecha.

Los datos de cada columna son numéricos, la celda está vacía si el alumno no se presentó a rendir. La nota mínima para aprobar cada parcial es 4, la materia se promociona con un promedio mayor o igual a 6. Contienen números tipo `float` entre 0 y 10.

- **F1**: columna numérica, nota del Flotante de Primer Parcial. Esta fecha permite recuperar el Primer Parcial si ya se logró aprobar el Segundo en las instancias anteriores. Contiene números tipo `float` entre 0 y 10.
- **F2**: columna numérica, nota del Flotante de Segundo Parcial. Esta fecha permite recuperar el Segundo Parcial si ya se logró aprobar el Primero en las instancias anteriores.  Contiene números tipo `float` entre 0 y 10.
- **Condicion**: columna categórica, indica la condición final del alumno luego de los parciales.
    -  *Libre*: no se presentó a ningún parcial.
    -  *Abandonó*: rindió algún parcial pero no agotó las instancias posibles.
    -  *Desaprobado*: no logró aprobar Primer Parcial y Segundo Parcial luego de agotar las instancias posibles, o sí lo hizo pero no logró alcanzar el promedio de Promoción.
    -  *Promocionado*: aprobó ambos parciales con un promedio mayor o igual a 6.
- **Final**: columna numérica, nota final de los alumnos promocionados, la celda está vacía en los otros casos.  Contiene números tipo `int` entre 6 y 10.
- **Año**: columna numérica, año de la cursada. Inicia en 2020.
- **Tipo_Cursada**: columna categórica, indica el período en el que se realizó la cursada.
    - *Verano*: cursada intensiva de Enero-Febrero.
    - *1er Semestre*: cursada regular de Marzo-Junio.
    - *Anticipada*: cursada regular de Agosto-Noviembre.
- **Virtual**: columna binaria, indica si la cursada fue dictada en modalidad virtual o no.
- **Oral**: columna binaria. En las cursadas virtuales, además de aprobar los parciales, se requería aprobar un examen oral. Esta columna indica los resultados como *Aprobado* o *Desaprobado*, la celda está vacía si el alumno no se presentó.
- **Grupo**: columna de texto, indica nombre de la comisión de la que se extrajeron los datos. Si la comisión es única la celda está vacía.

# Limpieza de los datos

Se realiza una revisión de cada columna, para asegurar la consistencia de los datos, corregir errores de tipeo o de tipo de dato.

## Revisión de tipo de datos y valores nulos

In [3]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1223 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          793 non-null    float64 
 1   1P2F          493 non-null    float64 
 2   2P1F          582 non-null    float64 
 3   2P2F          310 non-null    float64 
 4   F1            80 non-null     float64 
 5   F2            77 non-null     float64 
 6   Condicion     1223 non-null   category
 7   Final         347 non-null    float64 
 8   Año           1223 non-null   int64   
 9   Tipo_Cursada  1223 non-null   category
 10  Virtual       1223 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(7), int64(1), object(3)
memory usage: 117.4+ KB


### 1- Columna `Condicion`: revisión de consistencia

Debe corresponder con:

- Libre: si todas las columnas de Parciales y Flotante son nulas.
- Desaprobado: si el promedio entre 1P y 2P es menor que 6 y
    - F1 y F2 son nulas
    - F1 no nula y el promedio entre F1 y P1 es menor que 6
    - F2 no nula y el promedio entre F2 y P2 es menor que 6
- Promocionado: en otro caso 

In [4]:
# Revisión: Libre
libres = matepi[ matepi["Condicion"]=="Libre" ]

# Cantidad de nulos en cada columna: deben ser 318 registros
libres.isna().sum()

1P1F            318
1P2F            318
2P1F            318
2P2F            318
F1              318
F2              318
Condicion         0
Final           318
Año               0
Tipo_Cursada      0
Virtual           0
Oral            318
Grupo           249
dtype: int64

Se verifica que todas las columnas de Parciales y Flotantes son nulas para los Libres.

In [5]:
# Revision: Desaprobado
desaprobados = matepi[ matepi["Condicion"]=="Desaprobado" ]
desaprobados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.2,0.6,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1190,0.0,NaN,0.3,0.1,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1200,1.7,4.0,4.0,NaN,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1207,1.6,3.0,1.9,4.0,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN


Se ve que **hay filas que deberían corresponder con la categoría Libre**. Las aislamos:

In [6]:
# Error en la categoría: si todos los Parciales y Flotantes son nulos, debería ser Libre
desaprobados[desaprobados["1P1F"].isna() & desaprobados["1P2F"].isna() \
            & desaprobados["2P1F"].isna() & desaprobados["2P2F"].isna()]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
43,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN


In [7]:
# Los modifico para que correspondan a la categoría correcta:

condiciones = (matepi["Condicion"]=="Desaprobado") & matepi["1P1F"].isna() & matepi["1P2F"].isna() \
                & matepi["2P1F"].isna() & matepi["2P2F"].isna()

matepi.loc[condiciones, ["Condicion"]] = "Libre"

In [8]:
# Ya está corregido:
matepi[condiciones]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN
43,NaN,NaN,NaN,NaN,NaN,NaN,Libre,NaN,2020,Verano,No,NaN,NaN


**El error de categoría Desaprobado a Libre ya está corregido**. Puedo volver a hacer la revisión de consistencia respecto a parciales.

In [9]:
# Revision otra vez: Desaprobado
desaprobados = matepi[ matepi["Condicion"]=="Desaprobado" ]
desaprobados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
112,5.3,NaN,4.30,3.30,NaN,5.2,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN
116,4.3,4.7,6.10,NaN,5.5,NaN,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN
117,3.2,4.8,2.35,3.40,NaN,3.9,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN
128,3.2,6.6,0.30,0.00,NaN,5.4,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN
140,4.9,4.0,1.55,4.95,4.9,NaN,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.20,0.60,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1190,0.0,NaN,0.30,0.10,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1200,1.7,4.0,4.00,NaN,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1207,1.6,3.0,1.90,4.00,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN


In [10]:
# Ya que ya filtré que no hay libres, voy a reemplazar todos los nulos (ausentes) en 1P1F y 2P1F por 0:
parciales = ["1P1F", "2P1F"]
desaprobados[parciales].isna().sum()

1P1F    11
2P1F    16
dtype: int64

In [11]:
desaprobados.loc[:, parciales] = desaprobados.loc[:, parciales].fillna(0)

In [12]:
desaprobados[parciales].isna().sum()

1P1F    0
2P1F    0
dtype: int64

In [13]:
# Para las segundas fechas: si es nulo reemplazo por la nota en primera fecha
parciales_2 = ["1P2F", "2P2F"]
desaprobados[parciales_2].isna().sum()

1P2F    39
2P2F    22
dtype: int64

In [14]:
desaprobados.loc[:,"1P2F"] = desaprobados["1P2F"].fillna(desaprobados["1P1F"])
desaprobados.loc[:,"2P2F"] = desaprobados["2P2F"].fillna(desaprobados["2P1F"])

desaprobados[parciales_2].isna().sum()

1P2F    0
2P2F    0
dtype: int64

Voy a **verificar que los promedios sean menores que 6 antes del Flotante**:

In [15]:
# Agrego una columna con los promedios pre Flotante:
desaprobados.loc[:,"Promedio pre F"] = (desaprobados["1P2F"] + desaprobados["2P2F"])/2

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_13815/2713777927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desaprobados.loc[:,"Promedio pre F"] = (desaprobados["1P2F"] + desaprobados["2P2F"])/2


In [16]:
# Reviso que los promedios sean menores que 6:
desaprobados[ desaprobados["Promedio pre F"] >= 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F
147,6.0,6.00,6.35,6.35,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,6.175
171,8.6,8.60,8.90,8.90,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,8.750
189,8.2,8.20,6.70,6.70,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.450
193,7.6,7.60,3.80,7.20,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.400
214,9.0,9.00,5.50,5.50,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.250
217,7.0,7.00,9.50,9.50,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,8.250
236,3.8,5.42,7.00,7.00,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.210
238,6.8,6.80,6.80,6.80,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.800
261,6.9,6.90,7.20,7.20,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.050
275,7.6,7.60,6.20,6.20,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.900


Vemos que **existen promedios de Promoción previos al Flotante, pero desaprobaron en el Oral**.

Trabajemos con los que debían rendir Flotante (promedio menor que 6)

In [17]:
desaprobados[ desaprobados["Promedio pre F"] < 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F
112,5.3,5.3,4.30,3.30,NaN,5.2,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN,4.300
116,4.3,4.7,6.10,6.10,5.5,NaN,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN,5.400
117,3.2,4.8,2.35,3.40,NaN,3.9,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN,4.100
128,3.2,6.6,0.30,0.00,NaN,5.4,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,3.300
140,4.9,4.0,1.55,4.95,4.9,NaN,Desaprobado,NaN,2020,Anticipada,Si,NaN,NaN,4.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.20,0.60,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,1.000
1190,0.0,0.0,0.30,0.10,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,0.050
1200,1.7,4.0,4.00,4.00,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,4.000
1207,1.6,3.0,1.90,4.00,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,3.500


Reemplazo los valores nulos en F1 o F2 con la última nota obtenida en Primer Parcial o Segundo Parcial, para calcular el promedio final y **verificar que la nota final es menor que 6**.

In [18]:
# Reemplazo los nulos de Flotante con las últimas notas correspondientes:
desaprobados.loc[:,"F1"] = desaprobados["F1"].fillna(desaprobados["1P2F"])
desaprobados.loc[:,"F2"] = desaprobados["F2"].fillna(desaprobados["2P2F"])

# Calculo los promedios finales:
desaprobados.loc[:,"Promedio Final"] = (desaprobados["F1"] + desaprobados["F2"])/2

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_13815/3613982398.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desaprobados.loc[:,"Promedio Final"] = (desaprobados["F1"] + desaprobados["F2"])/2


In [19]:
# Reviso que los promedios sean menores que 6
desaprobados[ desaprobados["Promedio Final"] >= 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F,Promedio Final
128,3.2,6.60,0.30,0.00,6.60,5.40,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,3.300,6.000
147,6.0,6.00,6.35,6.35,6.00,6.35,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,6.175,6.175
165,4.5,4.50,5.20,5.80,6.70,5.80,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,5.150,6.250
166,5.7,5.70,3.20,0.00,5.70,7.45,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,2.850,6.575
171,8.6,8.60,8.90,8.90,8.60,8.90,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,8.750,8.750
189,8.2,8.20,6.70,6.70,8.20,6.70,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.450,7.450
193,7.6,7.60,3.80,7.20,7.60,7.20,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.400,7.400
214,9.0,9.00,5.50,5.50,9.00,5.50,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.250,7.250
217,7.0,7.00,9.50,9.50,7.00,9.50,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,8.250,8.250
229,4.6,6.83,5.00,3.80,6.83,6.80,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,5.315,6.815


Nuevamente, hay alumnos que **alcanzaron promedio de promoción pero desaprobaron en el Oral**

**Finalizamos la revisión de consistencia de Desaprobados.**

In [20]:
# Cantidad total de alumnos según Condicion
matepi.Condicion.value_counts()

Condicion
Abandonó        352
Promocionado    347
Libre           332
Desaprobado     192
Name: count, dtype: int64

Resta **analizar que los Promocionados sean correctos**.

In [24]:
# Filtro según Condicion Promocionado:
promocionados = matepi[ matepi["Condicion"] == "Promocionado" ]
promocionados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
1,NaN,7.0,8.2,NaN,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN
7,NaN,NaN,NaN,4.2,7.8,NaN,Promocionado,6.0,2020,Verano,No,NaN,NaN
10,8.6,NaN,7.9,NaN,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN
11,7.5,NaN,7.1,NaN,NaN,NaN,Promocionado,7.0,2020,Verano,No,NaN,NaN
12,NaN,8.3,9.5,NaN,NaN,NaN,Promocionado,9.0,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,3.3,5.0,4.6,7.0,NaN,NaN,Promocionado,6.0,2025,1er Semestre,No,NaN,NaN
1212,4.0,6.3,3.2,7.9,NaN,NaN,Promocionado,7.0,2025,1er Semestre,No,NaN,NaN
1216,7.5,6.7,NaN,NaN,NaN,7.5,Promocionado,7.0,2025,1er Semestre,No,NaN,NaN
1218,4.5,NaN,3.6,7.6,NaN,NaN,Promocionado,6.0,2025,1er Semestre,No,NaN,NaN


Igual que antes, voy a calcular los promedios antes del Flotante reemplazando valores nulos de segundas fechas con las notas correspondientes de las primeras

### 2- Columnas de parciales y flotantes

Hay muchos valores nulos, correspondientes a alumnos que no se presentaron a rendir. Como convención:

- Para **Primer y Segundo Parcial**:
    - Ausente: asignar valor -1
    - Rindió: valores `float` mayores o iguales a 0.
- Para **Flotantes**:
    - Ya estaba aprobado: asignar valor -2
    - Ausente: asignar valor -1
    - Rindió: valores `float` mayores o iguales a 0. 

In [21]:
parciales = ["1P1F", "1P2F", "2P1F", "2P2F"]
flotantes = ["F1", "F2"]

In [22]:

# Completar valores nulos con -1 en las columnas de Parciales:
#matepi.loc[:,parciales] = matepi.loc[:,parciales].fillna(-1)


In [23]:
matepi[parciales + flotantes + ["Condicion"]]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion
0,NaN,NaN,NaN,NaN,NaN,NaN,Libre
1,NaN,7.0,8.2,NaN,NaN,NaN,Promocionado
2,NaN,NaN,NaN,NaN,NaN,NaN,Libre
3,NaN,NaN,NaN,NaN,NaN,NaN,Libre
4,NaN,NaN,NaN,NaN,NaN,NaN,Libre
...,...,...,...,...,...,...,...
1218,4.5,NaN,3.6,7.6,NaN,NaN,Promocionado
1219,2.8,4.0,3.2,6.5,5.5,NaN,Promocionado
1220,1.0,1.2,NaN,NaN,NaN,NaN,Abandonó
1221,1.2,0.9,0.0,NaN,NaN,NaN,Abandonó
